#### **LangGraph - directory loader, history aware, streaming, faiss-index**

In [1]:
pip install langchain_community langchain langchain_huggingface langchain-core langgraph langchain_google_genai pypdf faiss-cpu

INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.4 MB/s 

In [1]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

In [3]:
import os
from typing import TypedDict, List, Literal
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document


# Configuration
DEFAULT_PDF_DIRECTORY = "/content/data"
FAISS_INDEX_PATH = "./faiss_index"


# Enhanced State Definition
class AgentState(TypedDict):
    messages: list
    mode: str
    pdf_directory: str
    response: str
    source_documents: List[Document]


# Initialize LLM with streaming enabled
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,
    api_key=api_key,
    #max_output_tokens= 1000,
    streaming=True
)


# Initialize embeddings globally
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


# Node 1: Direct LLM
def direct_llm_node(state: AgentState):
    """Direct interaction with LLM with conversation history"""
    messages = state["messages"]
    response = llm.invoke(messages)

    return {
        "response": response.content,
        "messages": messages + [AIMessage(content=response.content)],
        "source_documents": []
    }


# Node 2: RAG Pipeline with conversation history
def rag_pipeline_node(state: AgentState):
    """Complete RAG pipeline with conversation history support"""
    messages = state["messages"]
    current_query = messages[-1].content

    pdf_directory = state.get("pdf_directory", DEFAULT_PDF_DIRECTORY) or DEFAULT_PDF_DIRECTORY

    # Load or create FAISS index
    if os.path.exists(FAISS_INDEX_PATH):
        print("Loading existing FAISS index...")
        vectorstore = FAISS.load_local(
            FAISS_INDEX_PATH,
            embeddings,
            allow_dangerous_deserialization=True
        )
    else:
        print(f"Creating new FAISS index from directory: {pdf_directory}")

        # Load all PDFs from directory
        loader = PyPDFDirectoryLoader(
            path=pdf_directory,
            glob="**/*.pdf",  # Load all PDFs recursively
            recursive=True,    # Search subdirectories
            silent_errors=False  # Show errors if any
        )

        documents = loader.load()
        print(f"✓ Loaded {len(documents)} pages from PDF files in {pdf_directory}")

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=150
        )
        chunks = text_splitter.split_documents(documents)
        print(f"✓ Created {len(chunks)} chunks from documents")

        vectorstore = FAISS.from_documents(chunks, embeddings)
        vectorstore.save_local(FAISS_INDEX_PATH)
        print(f"✓ FAISS index saved to {FAISS_INDEX_PATH}")

    # Retrieval
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    relevant_docs = retriever.invoke(current_query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    # Build conversation history for context
    conversation_history = []
    for msg in messages[:-1]:  # Exclude current query
        if isinstance(msg, HumanMessage):
            conversation_history.append(f"User: {msg.content}")
        elif isinstance(msg, AIMessage):
            conversation_history.append(f"Assistant: {msg.content}")

    history_text = "\n".join(conversation_history) if conversation_history else "No previous conversation."

    # Enhanced prompt with conversation history
    prompt = f"""Based on the conversation history and context below, answer the current question.

Conversation History:
{history_text}

Context from documents:
{context}

Current Question: {current_query}

Answer (be specific and helpful):"""

    response = llm.invoke(prompt)

    return {
        "response": response.content,
        "messages": messages + [AIMessage(content=response.content)],
        "source_documents": relevant_docs
    }


# Routing Function
def route_query(state: AgentState) -> Literal["direct_llm", "rag_pipeline"]:
    """Route based on mode"""
    if state["mode"] == "rag":
        return "rag_pipeline"
    return "direct_llm"


# Build Graph
workflow = StateGraph(AgentState)
workflow.add_node("direct_llm", direct_llm_node)
workflow.add_node("rag_pipeline", rag_pipeline_node)

workflow.add_conditional_edges(
    START,
    route_query,
    {
        "direct_llm": "direct_llm",
        "rag_pipeline": "rag_pipeline"
    }
)

workflow.add_edge("direct_llm", END)
workflow.add_edge("rag_pipeline", END)

# Compile
app = workflow.compile()


#  CHAT FUNCTION
conversation_history = []


def chat(query: str, mode: str = "rag", pdf_directory: str = ""):
    """
    Chat function that maintains conversation history

    Args:
        query: User's question
        mode: "rag" or "direct"
        pdf_directory: Path to directory containing PDFs (empty for default)

    Returns:
        dict with response and source_documents
    """
    # Add user message to history
    conversation_history.append(HumanMessage(content=query))

    # Invoke with full conversation history
    result = app.invoke({
        "messages": conversation_history,
        "mode": mode,
        "pdf_directory": pdf_directory,
        "response": "",
        "source_documents": []
    })

    # Add AI response to history
    conversation_history.append(AIMessage(content=result["response"]))

    return result


def reset_conversation():
    """Clear conversation history to start fresh"""
    global conversation_history
    conversation_history = []
    print("🔄 Conversation history cleared!")


def reset_faiss_index():
    """Delete FAISS index to force recreation from PDFs"""
    import shutil
    if os.path.exists(FAISS_INDEX_PATH):
        shutil.rmtree(FAISS_INDEX_PATH)
        print(f"🗑️ FAISS index deleted from {FAISS_INDEX_PATH}")
    else:
        print("⚠️ No FAISS index found to delete")


In [ ]:
# Example 1 - Direct LLM
print("=" * 50)
print("Example 1: Direct LLM Mode")
print("=" * 50)

result0 = chat("What is Generative AI", mode="direct")

print("\n🤖 Response: ", end="")
for char in result0["response"]:
    print(char, end="", flush=True)
print("\n")

Example 1: Direct LLM Mode

🤖 Response: Of course! Here is a comprehensive explanation of Generative AI, broken down for easy understanding.

### The Simple Analogy: The Creative Apprentice

Imagine you have an apprentice who wants to learn to paint. You don't give them a set of rules like "a tree has a brown trunk and green leaves." Instead, you show them **millions of paintings** from every artist and every era.

The apprentice studies all of them—the brushstrokes of Van Gogh, the realism of Rembrandt, the abstracts of Picasso. After seeing all this art, they don't just copy one painting. They learn the underlying **patterns, styles, and relationships** that make up "art."

Now, you can ask the apprentice to create something new: "Paint me a picture of an astronaut riding a horse in the style of Van Gogh."

The apprentice, using everything it has learned, generates a **brand new, original painting** that fits your request. It's not a copy; it's a new creation based on a deep understa

In [ ]:
# Example 2 - Direct LLM - follow-up question
print("=" * 50)
print("Example 2: Direct LLM Follow-up")
print("=" * 50)

result00 = chat("How it was different from traditional machine learning?", mode="direct")

print("\n🤖 Response: ", end="")
for char in result00["response"]:
    print(char, end="", flush=True)
print("\n")

In [ ]:
# Example 3: First RAG query
print("=" * 50)
print("Example 3: RAG Mode (Multiple PDFs)")
print("=" * 50)

result1 = chat("What is Beam Search?", mode="rag")

print("\n🤖 Response: ", end="")
for char in result1["response"]:
    print(char, end="", flush=True)

print("\n\n--- Source Documents ---")
for idx, doc in enumerate(result1["source_documents"], 1):
    print(f"\n📄 Source {idx}:")
    print(f"   Page: {doc.metadata.get('page', 'N/A')}")
    print(f"   File: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:250]}...")

In [ ]:
# Example 4: Follow-up question (uses conversation history!)
print("\n" + "=" * 50)
print("Example 4: Follow-up Question")
print("=" * 50)

result2 = chat("How it is different from other models?", mode="rag")

print("\n🤖 Response: ", end="")
for char in result2["response"]:
    print(char, end="", flush=True)
print("\n")

print("\n--- Source Documents ---")
for idx, doc in enumerate(result2["source_documents"], 1):
    print(f"\n📄 Source {idx}:")
    print(f"   Page: {doc.metadata.get('page', 'N/A')}")
    print(f"   File: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:250]}...")